# Surface slabs

Generating surface slabs and related structures is a common task in computational materials science.

## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.ase.espresso.slabs import slabs_workgraph

task = slabs_workgraph.task()
task.to_html()
#
# task

NodeGraphWidget(settings={'minmap': False}, style={'width': '80%', 'height': '600px'}, value={'name': 'slabs_w…

## Visualizing the WorkGraph

In [1]:
from workgraph_collections.ase.espresso.slabs import slabs_workgraph

wg = slabs_workgraph()
wg.to_html()
# wg

## Example: Al (111) and Al (100) surfaces

### Prepare the inputs and submit the workflow


In [2]:
from workgraph_collections.ase.espresso.slabs import slabs_workgraph
from ase.build import bulk
from aiida import load_profile

load_profile()

# create input structure
atoms = bulk("Al")
# atoms = bulk("Si")

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}
# pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "CONTROL": {"calculation": "scf"},
    "SYSTEM": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
#------------------------- Set the inputs -------------------------
wg = slabs_workgraph(atoms=atoms,
                   computer="localhost",
                   miller_indices=[[1, 1, 1], [1, 0, 0]],
                   command="mpirun -np 4 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kspacing=0.1,
                   metadata=metadata)
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=300)



WorkGraph process created, PK: 154917


<WorkChainNode: uuid: 400b690c-40b7-4c00-8aa7-21ec90033533 (pk: 154917) (aiida.workflows:workgraph.engine)>

## Relaxed slabs

In [25]:
print("Surface   Energy")
for key, parameter in wg.tasks["relax_slabs"].outputs.parameters.value.items():
    print(key, parameter.value["energy"])

print("Surfae   Atoms")
for key, slab in wg.tasks["relax_slabs"].outputs.structures.value.items():
    print(key, slab.value)



Surface   Energy
111 -1611.6069376127
100 -1611.6069350276
Surfae   Atoms
111 Atoms(symbols='Al3', pbc=True, cell=[[2.8038215989776, -5.1940668363646e-18, 0.0], [1.4019107994888, 2.4281807323942, 0.0], [0.0, 0.0, 14.6373064841]])
100 Atoms(symbols='Al3', pbc=True, cell=[[2.8038334564601, 3.4727431156223e-18, 0.0], [1.4019167282301, 2.4281910012753, 0.0], [0.0, 0.0, 14.637152415272]])
